<h4>Mironov Mikhail. Master Thesis. Main research notebook</h4>

In [1]:
import warnings
from datetime import date, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from core.currency import Currency
from core.exchange import Exchange
from core.time_utils import Bounds
from core.utils import configure_logging
from ml_base.features import FeatureFilter, get_importance_file_path
from ml_base.metrics import log_lgbm_iteration_to_stdout
from typing import *

warnings.simplefilter("ignore")

configure_logging()

In [2]:
# Load features that are the most impactful
feature_filter: FeatureFilter = FeatureFilter.from_importance(
    get_importance_file_path(day=date(2025, 5, 25), target_exchange=Exchange.BINANCE_SPOT),
    use_first=25
)

feature_filter.allowed_features[:10]

2025-05-26 17:26:03,521 | INFO | root | Loading feature importance file D:\microstructure_ranker\src\models\prediction\feature_importances\BINANCE_SPOT-importances@20250525.csv


['SELF-flow_imbalance-2S@BINANCE_USDM',
 'ETH-asset_return-5S@BINANCE_SPOT',
 'SELF-asset_return-5S@BINANCE_SPOT',
 'currency_index',
 'SELF-flow_imbalance-2S@BINANCE_SPOT',
 'SELF-flow_imbalance-1S@BINANCE_USDM',
 'BTC-asset_return-1S@BINANCE_SPOT',
 'BTC-asset_return-500MS@BINANCE_SPOT',
 'SELF-share_of_long_trades-1S@BINANCE_USDM',
 'ETH-asset_return-2S@BINANCE_SPOT']

<h4>Define bounds for TRAIN and TEST samples</h4>

In [ ]:
train_bounds: Bounds = Bounds.for_days(
    date(2025, 4, 1), date(2025, 5, 5)
)

<h4>Build model manually</h4>


<p>Using BuildDataset read all features and split them into TRAIN and VALIDATION samples</p>

In [ ]:
from core.currency import get_target_currencies
from models.prediction.build_sample import BuildDataset
from ml_base.sample import SampleParams, Sample, MLDataset
from ml_base.enums import DatasetType

sample: Sample = (
    BuildDataset(
        target_exchange=Exchange.OKX_SPOT,
        feature_filter=feature_filter,
        target_currencies=get_target_currencies(),
        forecast_step=timedelta(seconds=5),
    )
    .create_sample(
        bounds=train_bounds,
        sample_params=SampleParams(train_share=.8, validation_share=.2),
    )
)

<h4>Visualize data</h4>

<p>We can get DataFrame from MLDataset</p>

In [ ]:
df_train = sample.get_data(ds_type=DatasetType.TRAIN)
df_val = sample.get_data(ds_type=DatasetType.VALIDATION)

df_train.shape, df_val.shape

<p>In MLDataset.eval_fields we have stored asset_hold_time. Now we will check what was the actual time between trades used to compute returns for different time horizons</p>

In [ ]:
train_eval: pd.DataFrame = sample.get_eval_data(ds_type=DatasetType.TRAIN)
val_eval: pd.DataFrame = sample.get_eval_data(ds_type=DatasetType.VALIDATION)

In [ ]:
currencies: List[Currency] = [Currency.BTC, Currency.ETH, Currency.HBAR]
vals: List[pd.Series] = []

for currency in currencies:
    mask = df_train["currency_index"] == currency.value
    vals.append(train_eval[mask].mean())

In [ ]:
# Get hold_time for different windows for less traded currency like HBAR
df_liquidity = pd.DataFrame(vals).T
df_liquidity.columns = [currency.name for currency in currencies]


df_liquidity

In [ ]:
df_train.sample(int(5 * 1e6), replace=False).describe().T

<h4>Train the model with early stopping on the validation sample</h4>

In [ ]:
# Now train the model using LightGBM
from lightgbm import Booster, record_evaluation
from typing import *
import lightgbm as lgb
import os

_BASE_PARAMS: Dict[str, Any] = {
    "objective": "mse",
    "max_depth": 10,
    "learning_rate": 0.05,
    "n_estimators": 120,
    "subsample": 0.7,
    "num_threads": os.cpu_count() - 1,
    "verbose": -1
}

evals_result = {}

train: lgb.Dataset = sample.get_lgb_dataset(ds_type=DatasetType.TRAIN)
validation: lgb.Dataset = sample.get_lgb_dataset(ds_type=DatasetType.VALIDATION)

booster: Booster = lgb.train(
    params=_BASE_PARAMS,
    train_set=train,
    valid_sets=[train, validation],
    valid_names=["train", "validation"],
    callbacks=[
        record_evaluation(evals_result),
        lgb.early_stopping(stopping_rounds=50, verbose=False),
        log_lgbm_iteration_to_stdout
    ]
)

In [ ]:
fig, ax1 = plt.subplots()

ax1.set_xlabel('Iteration')
ax1.plot(evals_result["train"]["l2"], label='Train L2', color="red")

# create a second y-axis sharing the same x
ax2 = ax1.twinx()
ax2.plot(evals_result["validation"]["l2"], label='Validation L2', color="blue")

fig.legend()

<h4>Evaluate the model</h4>

In [ ]:
val_ds: MLDataset = sample.get_dataset(ds_type=DatasetType.VALIDATION)
y_pred: np.ndarray = booster.predict(val_ds.data, num_iteration=booster.best_iteration)

In [ ]:
from sklearn.metrics import r2_score, classification_report, accuracy_score

r2_score(y_pred=y_pred, y_true=val_ds.label)

In [ ]:
y_pred_binary: np.ndarray = (y_pred > 0).astype(int)
y_true_binary: np.ndarray = (val_ds.label > 0).astype(int)

print(
    classification_report(y_pred=y_pred_binary, y_true=y_true_binary)
)

In [ ]:
accuracy_score(y_pred=y_pred_binary, y_true=y_true_binary)

In [ ]:
# display r2 by currency
from ml_base.metrics import compute_metrics


compute_metrics(
    booster=booster, dataset=val_ds, target_currencies=get_target_currencies()
)